# Saving data to netCDF

xarray supports saving data to a number of formats, but this tutorial will only cover saving to netCDF files.

The most common usage pattern for saving data is one where data is read from a netCDF file, manipulated or analysed in some way and then written out again. This is what is assumed in this tutorial.

For more information refer to the excellent xarray documentation on [reading and writing files](http://xarray.pydata.org/en/stable/io.html)

In [1]:
import xarray

Load the surface air temperature dataset that has been used previously

In [2]:
url = 'http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc'
ds = xarray.open_dataset(url)
tas = ds.tas

In [3]:
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 145, lon: 192, time: 1872)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2005-12-16T12:00:00
  * lat        (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes:
    institution:                     CSIRO (Commonwealth Scientific and Indus...
    institute_id:                    CSIRO-BOM
    experiment_id:                   historical
    source:                          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N...
    model_id:                        ACCESS1.3
    forcing:                         GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2,...
    parent_experiment_id:  

Because xarray saves all the metadata and coordinates in a dataset object it is very simple to write it back out to a netCDF file. To save a dataset to a local file call the `to_netcdf` method and supply a filename

In [4]:
ds.to_netcdf('ds.nc')

And that is it. The saved file can be read back into a new variable, `ds_local` and inspection of the metadata suggests it is the same. It is impossible to say for certain just by inspecting the metadata, luckily ...

In [5]:
ds_local = xarray.open_dataset('ds.nc')

In [6]:
ds_local

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 145, lon: 192, time: 1872)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2005-12-16T12:00:00
  * lat        (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes:
    institution:                     CSIRO (Commonwealth Scientific and Indus...
    institute_id:                    CSIRO-BOM
    experiment_id:                   historical
    source:                          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N...
    model_id:                        ACCESS1.3
    forcing:                         GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2,...
    parent_experiment_id:  

There is an operator, `identical`, that will compare datasets and return `True` if they are identical. Running that shows that yes, xarray has saved the dataset read from an opendap URL to a local file and `ds` and `ds_local` are identical

In [7]:
ds.identical(ds_local)

True

The same `to_netcdf` operator will work for a `DataArray`

In [8]:
tas.to_netcdf('tas.nc')

When this is read back in and printed out none of the metadata from the original dataset, `ds`, is retained as it did not exist in the `tas` `DataArray`

In [9]:
tas_local = xarray.open_dataset('tas.nc')
tas_local

<xarray.Dataset>
Dimensions:  (lat: 145, lon: 192, time: 1872)
Coordinates:
  * time     (time) datetime64[ns] 1850-01-16T12:00:00 ... 2005-12-16T12:00:00
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
    height   float64 ...
Data variables:
    tas      (time, lat, lon) float32 ...

Using the `identical` method shows the newly created `tas` `Dataset` is not identical to the `DataArray` `tas`.

In [10]:
tas.identical(tas_local)

False

If the comparison is between `tas` and the `DataArray` `tas` inside the `tas_local` `Dataset` it shows they are identical

In [11]:
tas.identical(tas_local.tas)

True

A `DataArray` cannot be written directly to a netCDF file. When `to_netcdf` is called on a `DataArray` there is an implicit `to_dataset` operation then `to_netcdf` is called on the resulting `DataSet`

In [12]:
tas.to_dataset()

<xarray.Dataset>
Dimensions:  (lat: 145, lon: 192, time: 1872)
Coordinates:
  * time     (time) datetime64[ns] 1850-01-16T12:00:00 ... 2005-12-16T12:00:00
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
    height   float64 1.5
Data variables:
    tas      (time, lat, lon) float32 240.83618 240.83618 ... 246.2805 246.2805

So the result of `tas.to_dataset()` is identical to `tas_local`, as this is what was done implicitly before `to_netcdf`

In [13]:
tas.to_dataset().identical(tas_local)

True

When xarray opens a netCDF file some of the metadata is saved in a special `dict` called `encoding`. This is used when xarray writes a `Dataset` back out to a netCDF file, it encodes the data before doing so. There is an `encoding` attribute for a `Dataset` and all variables and coordinates have their own `encoding` attribute:

In [14]:
ds.encoding

{'unlimited_dims': {'time'},
 'source': 'http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc'}

In [15]:
tas.encoding

{'source': 'http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc',
 'original_shape': (1872, 145, 192),
 'dtype': dtype('float32'),
 'missing_value': 1e+20,
 '_FillValue': 1e+20,
 'coordinates': 'height'}

The `encoding` `dict` for the `time` coordinate is particularly interesting, as this is where the `calendar` and `units` attributes from the netCDF file are stored. When the netCDF file is opened these attributes are used to convert the time variable, which is just an array of numbers, into python `DateTime` objects. When the data is written back to a file this step must be done in reverse

In [16]:
ds.time.encoding

{'source': 'http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc',
 'original_shape': (1872,),
 'dtype': dtype('float64'),
 'units': 'days since 0001-01-01',
 'calendar': 'proleptic_gregorian'}

netCDF4 files support lossless in-file compression. This can reduce the size of a netCDF file considerably, but it can still be used in the same way as an uncompressed netDF file. See the CMS Wiki for [more details in netCDF compression](http://climate-cms.wikis.unsw.edu.au/NetCDF_Compression_Tools).

Enabling in-file compression when saving data to a netCDF file is done by adding the appropriate key/value pairs to the `encoding`. The `encoding` can be updated in the object before writing to a file, or an `encoding` argument added to the `to_netcdf` method. The latter is shown below for the the `tas` `DataArray`. It is sufficient to just specify `zlib`, which would use the default compression level, `complevel`, of 1.

In [17]:
tas.to_netcdf('tas_compressed.nc', encoding=tas.encoding.update({'zlib': True, 'complevel': 4}))

A directory listing shows that the compressed file is 56% the size of the uncompressed data, a 1.8 compression factor. 3D ocean data will typically have compression ratios of 3-4.

In [18]:
!ls -lh tas.nc tas_compressed.nc

-rw-r--r-- 1 aph502 v45 111M Jul 25 14:53 tas_compressed.nc
-rw-r--r-- 1 aph502 v45 199M Jul 25 14:53 tas.nc


Reading the compressed data back in shows the compression was lossless and the data is unchanged

In [19]:
tas.identical(xarray.open_dataset('tas_compressed.nc').tas)

True

The same `encoding` option works for `Dataset` objects also

In [20]:
ds.to_netcdf('ds_compressed.nc', encoding=ds.encoding.update({'zlib': True, 'complevel': 4}))

In [21]:
!ls -lh ds.nc ds_compressed.nc

-rw-r--r-- 1 aph502 v45 113M Jul 25 14:53 ds_compressed.nc
-rw-r--r-- 1 aph502 v45 200M Jul 25 14:53 ds.nc


To save new datasets, say the results of calculations, is the same process. As an example from the surface air temp data, `tas`, a 30 year climatology is calculated and saved in the `tas_clim` variable

In [22]:
tas_clim = tas.sel(time=slice('1960-01','1989-12')).mean(dim='time')

In [23]:
tas_clim

<xarray.DataArray 'tas' (lat: 145, lon: 192)>
array([[223.5951 , 223.5951 , 223.5951 , ..., 223.58675, 223.58675, 223.58675],
       [227.05313, 227.00461, 226.95636, ..., 227.20116, 227.1515 , 227.10295],
       [228.13997, 228.0243 , 227.9114 , ..., 228.49681, 228.37773, 228.25885],
       ...,
       [255.42877, 255.46603, 255.50294, ..., 255.29857, 255.34636, 255.38702],
       [254.99307, 255.0079 , 255.0223 , ..., 254.94296, 254.95801, 254.97554],
       [254.3218 , 254.3218 , 254.3218 , ..., 254.3218 , 254.3218 , 254.3218 ]],
      dtype=float32)
Coordinates:
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
    height   float64 1.5

Now this is saved to the netCDF file `tas_clim.nc`. Note that this is calling `to_dataset` explicitly so that a new name can be specified for the `DataArray` when it is converted into a `Dataset`

In [24]:
tas_clim.to_dataset(name='tas_climatology').to_netcdf('tas_clim.nc')

Open the data again, overwriting the `tas_clim` variable in the process, and the data variable is now named `tas_climatology` as expected. If the name had not been defined it would have used the name from the original `DataArray`, namely `tas`

In [25]:
tas_clim = xarray.open_dataset('tas_clim.nc')

In [26]:
tas_clim

<xarray.Dataset>
Dimensions:          (lat: 145, lon: 192)
Coordinates:
  * lat              (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 87.5 88.75 90.0
  * lon              (lon) float64 0.0 1.875 3.75 5.625 ... 354.4 356.2 358.1
    height           float64 ...
Data variables:
    tas_climatology  (lat, lon) float32 ...